In [18]:
from neo4j import GraphDatabase
from neo4j_graphrag.indexes import create_fulltext_index, create_vector_index,drop_index_if_exists

from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever, HybridCypherRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "h4ck3r20o8"))

api_key = "ollama"

embedder = OpenAIEmbeddings(
    base_url="http://localhost:11434/v1",
    api_key=api_key,
    model="snowflake-arctic-embed:latest",
)

llm = OpenAILLM(
    base_url="http://localhost:11434/v1",
    model_name="llama3",
    api_key=api_key,
)

In [2]:
FULLTEXT_INDEX_NAME = "fulltext_index"

create_fulltext_index(
    driver, FULLTEXT_INDEX_NAME, label="Pokemon", node_properties=["description"]
)

In [3]:
INDEX_NAME = "vector_index"
DIMENSION = 1024

drop_index_if_exists(driver,INDEX_NAME,)

create_vector_index(
    driver,
    INDEX_NAME,
    label="Pokemon",
    embedding_property="vector_property",
    dimensions=DIMENSION,
    similarity_fn="cosine",
)

In [4]:
retriever = VectorRetriever(
    driver,
    index_name="vector_index",
    embedder=embedder,
    return_properties=["name", "description"],
)

query_text = "Who is Pikachu"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

rag = GraphRAG(retriever=retriever, llm=llm)

# Query the graph
query_text = "What is the type and Species of Pikachu"
response = rag.search(query_text=query_text, retriever_config={"top_k": 5})
print(response.answer)

items=[RetrieverResultItem(content='{\'description\': "Pikachu is a beloved and iconic Pokémon character in the popular franchise created by Satoshi Tajiri. It\'s a small, rodent-like creature with yellow fur and red circles on its cheeks that can store electricity in its cheeks. Pikachu is known for its ability to deliver powerful electric attacks and is often considered one of the most popular and recognizable Pokémon characters.", \'name\': \'Pikachu\'}', metadata={'score': 0.9058599472045898, 'nodeLabels': ['Pokemon'], 'id': '4:c543a4c6-6e09-4edb-a68d-17f345c3ce0b:782'}), RetrieverResultItem(content='{\'description\': \'Pikachu is a beloved and iconic fictional creature from the popular "Pokémon" franchise. It is a small, rodent-like Pokémon with electric powers. Pikachu is known for its bright yellow color, red cheeks, and adorable appearance. In the franchise\\\'s storyline, Pikachu is the partner Pokémon of Ash Ketchum, the main protagonist, and has been depicted as one of the m

In [23]:
retrieval_query = """
    MATCH (t:Type)<-[:HAS_TYPE]-(node)
    MATCH (s:Species)<-[:BELONGS_TO]-(node)
    RETURN
        node.name AS pokemon_name,
        node.description AS pokemon_description, 
    collect(t.name) AS types, collect(s.name) as species;
"""

retrieval_query2 = """ 
MATCH (s:Species)<-[:BELONGS_TO]-(node)-[:HAS_TYPE]->(t:Type) 
return node.name as pokemon_name, node.description, collect(s.name) as species, collect(t.name) as types
"""
retriever = HybridCypherRetriever(
    driver,
    vector_index_name="vector_index",
    fulltext_index_name="fulltext_index",
    retrieval_query=retrieval_query2,
    embedder=embedder, 
)

query_text = "What is the type and Species of Pikachu"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)
response = rag.search(query_text=query_text, retriever_config={"top_k": 5})
print(response.answer)

items=[RetrieverResultItem(content="<Record pokemon_name='Partner Pikachu' species=['Mouse'] types=['Electric']>", metadata=None), RetrieverResultItem(content="<Record pokemon_name='Pichu' species=['Tiny Mouse'] types=['Electric']>", metadata=None), RetrieverResultItem(content="<Record pokemon_name='Pikachu' species=['Mouse'] types=['Electric']>", metadata=None)] metadata={'__retriever': 'HybridCypherRetriever'}
Pikachu's species is ['Mouse'], and its type is ['Electric'].


In [26]:
query_text = "List all pokemons that have the same type and species of Pikachu"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)
response = rag.search(query_text=query_text, retriever_config={"top_k": 3})
print(response.answer)

items=[RetrieverResultItem(content="<Record pokemon_name='Toxel' species=['Baby', 'Baby'] types=['Poison', 'Electric']>", metadata=None), RetrieverResultItem(content="<Record pokemon_name='Pichu' species=['Tiny Mouse'] types=['Electric']>", metadata=None), RetrieverResultItem(content="<Record pokemon_name='Pikachu' species=['Mouse'] types=['Electric']>", metadata=None)] metadata={'__retriever': 'HybridCypherRetriever'}
Based on the given context, since Pikachu is a 'Mouse' with the Electric type, we are looking for pokemons that also share these characteristics. 

Toxel shares the same Electric type as Pikachu, but its species is different (Baby instead of Mouse). Pichu does have the same Electric type as Pikachu, and both species ("Tiny Mouse" for Pichu and "Mouse" for Pikachu) belong to mammals, so we consider it a match.

Therefore, the answer would be: Pichu
